In [1]:
import cv2
import numpy as np
import re
import os

def pfm_imread(filename):
    file = open(filename, 'rb')
    color = None
    width = None
    height = None
    scale = None
    endian = None

    header = file.readline().decode('utf-8').rstrip()
    if header == 'PF':
        color = True
    elif header == 'Pf':
        color = False
    else:
        raise Exception('Not a PFM file.')

    dim_match = re.match(r'^(\d+)\s(\d+)\s$', file.readline().decode('utf-8'))
    if dim_match:
        width, height = map(int, dim_match.groups())
    else:
        raise Exception('Malformed PFM header.')

    scale = float(file.readline().rstrip())
    if scale < 0:  
        endian = '<'
        scale = -scale
    else:
        endian = '>'  

    data = np.fromfile(file, endian + 'f')
    shape = (height, width, 3) if color else (height, width)

    data = np.reshape(data, shape)
    data = np.flipud(data)
    return data, scale


In [3]:
mse_error, mde_error = 0, 0
# model_name = 'Original Model'
model_name = 'Train'
root = '/home/tlab4/cj/data_mvsec/data_train/VAL/left'
# lens = len(os.listdir(root))
lens = 3000
# search predictions
for i in range(3300):
    error = 0
    # dp1 = '/home/tlab4/cj/mobilestereonet/predictions_test/{}.pfm'.format(i)
    dp1 = '/home/tlab4/cj/mobilestereonet/predictions_train/{}.pfm'.format(i)
    # dp2 = '/home/tlab4/cj/data_mvsec/data_train/VAL/depth/{}.pfm'.format(i)
    dp2 = '/home/tlab4/cj/data_mvsec/data_train/TRAIN/depth/{}.pfm'.format(i)
    pred, _ = pfm_imread(dp1)
    gt, _ = pfm_imread(dp2)
    pred, gt = np.array(pred), np.array(gt)
    error1 = np.nanmean((gt - pred) ** 2) # MSE
    error2 = np.nanmean(abs(gt-pred)) # MDE (MAE)
    # print(mse)
    mse_error += error1
    mde_error += error2

avg_mse_error = mse_error / lens
avg_mde_error = mde_error / lens

print('the average MSE error is: ', avg_mse_error)
print('the average MDE error is: ', avg_mde_error)


with open('/home/tlab4/cj/data_mvsec/Error.txt', 'a') as f:
    f.writelines(model_name + ':\n')
    f.writelines('MSE: {}        MDE:{} \n\n'.format(avg_mse_error, avg_mde_error))

the average MSE error is:  0.1485025925754259
the average MDE error is:  0.16411261067787805
